In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_val_score, KFold
import seaborn as sns

In [18]:
def transform_features(data):
    
    # Dropping columns with too many missing values
    null_percent=data.isna().sum()/data.shape[0]
    drop_columns=null_percent[null_percent>0.05].index
    data=data.drop(columns=drop_columns,axis=1)
    
    # Dropping text columns with any missing values
    text_data=data.select_dtypes('object')
    null_text=text_data.isna().sum()
    drop_text_cols=null_text[null_text>0].index
    data=data.drop(columns=drop_text_cols,axis=1)
    
    # Filling in missing numeric data with the mode of that column
    numeric_data=data.select_dtypes('number')
    numeric_data=numeric_data.apply(lambda col:col.fillna(col.mode()[0]),axis=0)
    data[numeric_data.columns]=numeric_data
    
    # Create some new features like number of years on market and years since remod
    years_on_market=data['Yr Sold']-data['Year Built']
    years_since_remod = data['Yr Sold'] - data['Year Remod/Add']
    data['years_on_market']=years_on_market
    data['years_since_remod']=years_since_remod
    data=data.drop(columns=['Year Built','Year Remod/Add'],axis=1)
    # Remove rows where year_on_market or years_since_remod don't make since
    data.drop(data[(data['years_on_market']<0) | (data['years_since_remod']<0)].index,inplace=True)
    
    # Finally remove columns that are either not useful for the regression or leak data about the final sale
    data = data.drop(["PID", "Order"], axis=1)
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return data

In [28]:
def feature_selection(data,threshold=0.4,cutoff=10):
    
    #We only include numeric columns that correlate well with the SalePrice column
    abs_corr=data.corr()['SalePrice'].abs().sort_values(ascending=False)
    high_corr_data=data.drop(columns=abs_corr[abs_corr<threshold].index, axis=1)
    
    #We look at which text columns could be converted to categorical columns using dummy variable. 
    #We set a cutoff for how many unique values the column can have.
    object_columns=high_corr_data.select_dtypes('object').columns
    unique_values={col:len(high_corr_data[col].unique()) for col in object_columns}
    unique_values=pd.Series(unique_values)
    high_corr_data.drop(columns=unique_values[unique_values>cutoff].index,axis=1,inplace=True)
    
    #Now we'll convert the remaing object columns to dummy variables and add these columns to the data set
    object_cols=high_corr_data.select_dtypes('object').columns
    for col in object_cols:
        high_corr_data[col]=high_corr_data.astype('category')
        dummy_df=pd.get_dummies(high_corr_data[col],prefix=col)
        high_corr_data=pd.concat([high_corr_data,dummy_df],axis=1)
    high_corr_data.drop(columns=object_cols,axis=1,inplace=True)
    return high_corr_data

In [85]:
def train_and_test(data,k=0):
    numeric_data=data.select_dtypes('number')
    features=numeric_data.columns.drop('SalePrice')
    lr=LinearRegression()
    
    if k==0:
        #Perform simple hold-out validation
        train=numeric_data.iloc[:1460]
        test=numeric_data.iloc[1460:]
 
        lr.fit(train[features],train['SalePrice'])
        predictions=lr.predict(test[features])
        mse=mean_squared_error(test['SalePrice'],predictions)
        rmse=np.sqrt(mse)
        return rmse
    
    if k==1:
        # perform two-fold cross-validation
        shuffled_data=numeric_data.sample(frac=1, )
        fold_one=shuffled_data.iloc[:1460]
        fold_two=shuffled_data.iloc[1460:]
        
        lr.fit(fold_one[features],fold_one['SalePrice'])
        predictions_1=lr.predict(fold_two[features])
        mse_1=mean_squared_error(fold_two['SalePrice'],predictions_1)
        rmse_1=np.sqrt(mse_1)
        
        lr.fit(fold_two[features],fold_two['SalePrice'])
        predictions_2=lr.predict(fold_one[features])
        mse_2=mean_squared_error(fold_one['SalePrice'],predictions_2)
        rmse_2=np.sqrt(mse_2)
        
        return (rmse_1+rmse_2)/2
    
    if k>1:
        # perform k-fold cross validation
        kf=KFold(n_splits=k,shuffle=True)
        rmses=[]
        
        for train_index,test_index in kf.split(numeric_data):
            train=numeric_data.iloc[train_index]
            test=numeric_data.iloc[test_index]
            lr.fit(train[features],train['SalePrice'])
            predictions=lr.predict(test[features])
            mse=mean_squared_error(test['SalePrice'],predictions)
            rmse=np.sqrt(mse)
            rmses.append(rmse)
        return np.mean(rmses)

In [9]:
housing=pd.read_csv('AmesHousing.tsv',delimiter='\t')
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [20]:
data=transform_features(housing)
data

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_on_market,years_since_remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,210,62,0,0,0,0,0,215000,50,50
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,140,0,0,0,120,0,0,105000,49,49
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,393,36,0,0,0,0,12500,172000,52,52
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,0,0,0,0,0,0,0,244000,42,42
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,212,34,0,0,0,0,0,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,120,0,0,0,0,0,0,142500,22,22
2926,20,RL,8885,Pave,IR1,Low,AllPub,Inside,Mod,Mitchel,...,164,0,0,0,0,0,0,131000,23,23
2927,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,...,80,32,0,0,0,0,700,132000,14,14
2928,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,...,240,38,0,0,0,0,0,170000,32,31


In [30]:
good_data=feature_selection(data,threshold=0.4,cutoff=10)

In [63]:
good_data.isna().sum()

Overall Qual           0
Mas Vnr Area           0
BsmtFin SF 1           0
Total Bsmt SF          0
1st Flr SF             0
                      ..
Paved Drive_FV         0
Paved Drive_I (all)    0
Paved Drive_RH         0
Paved Drive_RL         0
Paved Drive_RM         0
Length: 168, dtype: int64

In [53]:
numeric_data=good_data.select_dtypes('number')
features=numeric_data.columns.drop('SalePrice')
lr=LinearRegression()

In [65]:
permutation=np.random.permutation(len(numeric_data))
numeric_data=numeric_data.reindex(permutation)
fold_one=numeric_data.iloc[:1460,:]
fold_two=numeric_data.iloc[1460:,:]

In [66]:
fold_one[features].isna().sum()

Overall Qual           2
Mas Vnr Area           2
BsmtFin SF 1           2
Total Bsmt SF          2
1st Flr SF             2
                      ..
Paved Drive_FV         2
Paved Drive_I (all)    2
Paved Drive_RH         2
Paved Drive_RL         2
Paved Drive_RM         2
Length: 167, dtype: int64

In [61]:
shuffled_df = good_data.sample(frac=1, )

In [67]:
shuffled_df.isna().sum()

Overall Qual           0
Mas Vnr Area           0
BsmtFin SF 1           0
Total Bsmt SF          0
1st Flr SF             0
                      ..
Paved Drive_FV         0
Paved Drive_I (all)    0
Paved Drive_RH         0
Paved Drive_RL         0
Paved Drive_RM         0
Length: 168, dtype: int64

In [86]:
train_and_test(good_data,k=2)

33081.31768588226